In [1]:
import time
import argparse
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import torch
import numpy as np
import cv2
from skimage.measure import compare_ssim, compare_psnr

import matplotlib.pyplot as plt
import utils
import dataset

def str2bool(v):
    #print(v)
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Unsupported value encountered.')

if __name__ == "__main__":
    # ----------------------------------------
    #        Initialize the parameters
    # ----------------------------------------
    parser = argparse.ArgumentParser()
    #GPU parameters
    parser.add_argument('--no_gpu', default = False, help = 'True for CPU')
    # Saving, and loading parameters
    
    #models_TinyDerainGAN_raindrop
    parser.add_argument('--save_name', type = str, default = '../results/models_SAUnet_SASkipnet_SAPerceptualVGG16SArelu53_raindrop_b', help = 'save the generated with certain epoch')
    parser.add_argument('--load_gname', type = str, default = '../models/models_SAUnet_SASkipnet_SAPerceptualVGG16SArelu53_raindrop/KPN_rainy_image_epoch200_bs8_generator.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--load_dname', type = str, default = '../models/models_TinyDerainGAN_raindrop/KPN_rainy_image_epoch190_bs4_discriminator.pth', help = 'load the pre-trained model with certain epoch')
    
    parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/raindrop/test_b', help = 'images baseroot')
    
    #models_SAUnet_DF_raindrop
    #parser.add_argument('--save_name', type = str, default = '../results/models_SAUnet_DF_raindrop_b', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = '../models/models_SAUnet_DF_raindrop/KPN_rainy_image_epoch200_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/raindrop/test_b', help = 'images baseroot')
    
    #models_SAUnet_DF_rain100H
    #parser.add_argument('--save_name', type = str, default = '../results/models_SAUnet_DF_rain100H', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = '../models/models_SAUnet_DF_rain100H/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/rain100H/test', help = 'images baseroot')
    
    #models_SA_onlySA
    #parser.add_argument('--save_name', type = str, default = './results/models_SA_onlySA', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SA_onlySA/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #models_SA_onlySA2conv_deconv_SAGAN
    #parser.add_argument('--save_name', type = str, default = './results/models_SA_onlySA2conv_deconv_SAGAN', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SA_onlySA2conv_deconv_SAGAN/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #models_Original_KPN
    #parser.add_argument('--save_name', type = str, default = './results/models_Original_KPN', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_Original_KPN/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #models_original_KPN_rainmix_rain100H
    #parser.add_argument('--save_name', type = str, default = './results/models_original_KPN_rainmix_rain100H', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_original_KPN_rainmix_rain100H/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = './rainy_image_dataset/rain100H/test/', help = 'images baseroot')
    
    #models_original_KPN_Rain1400
    #parser.add_argument('--save_name', type = str, default = './results/models_original_KPN_Rain1400', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_original_KPN_Rain1400/KPN_rainy_image_epoch100_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/Rain1400/test/', help = 'images baseroot')
    
    #models_SAUnet_SASkipnet4level_rain100H
    #parser.add_argument('--save_name', type = str, default = './results/models_SAUnet_SASkipnet4level_rain100H', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SAUnet_SASkipnet4level_rain100H/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/rain100H/test/', help = 'images baseroot')
    
    #models_SAUnet_SASkipnet_rain100H
    #parser.add_argument('--save_name', type = str, default = './results/models_SAUnet_SASkipnet_rain100H', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SAUnet_SASkipnet_rain100H/KPN_rainy_image_epoch250_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/rain100H/test/', help = 'images baseroot')
    
    #models_SAUnet_SASkipnet_notanh_Rain1400
    #parser.add_argument('--save_name', type = str, default = './results/models_SAUnet_SASkipnet_notanh_Rain1400', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SAUnet_SASkipnet_notanh_Rain1400/KPN_rainy_image_epoch100_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/Rain1400/test/', help = 'images baseroot')
    
    #models_SAUnet_skipnet_notanh_3conv_Rain1400
    #parser.add_argument('--save_name', type = str, default = './results/models_SAUnet_skipnet_notanh_3conv_Rain1400', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SAUnet_skipnet_notanh_3conv_Rain1400/KPN_rainy_image_epoch90_bs8.pth', help = 'load the pre-trained model with certain epoch')
    #parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/Rain1400/test/', help = 'images baseroot')
    
    #models_SA_onlySA2conv
    #parser.add_argument('--save_name', type = str, default = './results/models_SA_onlySA2conv', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SA_onlySA2conv/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #models_SA_onlySA2conv_deconv
    #parser.add_argument('--save_name', type = str, default = './results/models_SA_onlySA2conv_deconv', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SA_onlySA2conv_deconv/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #models_SA_onlySA2conv_deconv_SAGAN_bottom1024
    #parser.add_argument('--save_name', type = str, default = './results/models_SA_onlySA2conv_deconv_SAGAN_bottom1024', help = 'save the generated with certain epoch')
    #parser.add_argument('--load_name', type = str, default = './models/models_SA_onlySA2conv_deconv_SAGAN_bottom1024/KPN_rainy_image_epoch250_bs16.pth', help = 'load the pre-trained model with certain epoch')
    
    #parser.add_argument('--load_name', type = str, default = './models/KPN_single_image_epoch120_bs16_mu0_sigma30.pth', help = 'load the pre-trained model with certain epoch')
    parser.add_argument('--test_batch_size', type = int, default = 1, help = 'size of the batches')
    parser.add_argument('--num_workers', type = int, default = 1, help = 'number of workers')
    # Initialization parameters
    parser.add_argument('--init_type', type = str, default = 'xavier', help = 'initialization type of generator')
    parser.add_argument('--init_gain', type = float, default = 0.02, help = 'initialization gain of generator')
    # Dataset parameters
    parser.add_argument('--crop', type = str2bool, default = False, help = 'whether to crop input images')
    parser.add_argument('--crop_size', type = int, default = 512, help = 'single patch size')
    parser.add_argument('--resize', type = str2bool, default = True, help = 'whether to resize input images')
    parser.add_argument('--scale_size', type = int, default = 1024, help = 'single patch size')
    parser.add_argument('--geometry_aug', type = str2bool, default = False, help = 'geometry augmentation (scaling)')
    parser.add_argument('--angle_aug', type = str2bool, default = False, help = 'geometry augmentation (rotation, flipping)')
    parser.add_argument('--scale_min', type = float, default = 1, help = 'min scaling factor')
    parser.add_argument('--scale_max', type = float, default = 1, help = 'max scaling factor')
    parser.add_argument('--add_noise', type = str2bool, default = False, help = 'whether to add noise to input images')
    parser.add_argument('--mu', type = int, default = 0, help = 'Gaussian noise mean')
    parser.add_argument('--sigma', type = int, default = 30, help = 'Gaussian noise variance: 30 | 50 | 70')
    opt = parser.parse_args(args=[])
    print(opt)

    # ----------------------------------------
    #                   Test
    # ----------------------------------------
    # Initialize
    if opt.no_gpu:
        generator = utils.create_generator(opt)
        #discriminator = utils.create_discriminator(opt)
    else:
        generator = utils.create_generator(opt).cuda()
        #discriminator = utils.create_discriminator(opt).cuda()
    

    '''
    parm={}
    for name,parameters in generator.named_parameters():
        print(name,':',parameters.size())
        parm[name]=parameters.detach().cpu().numpy()
    print(parm['conv_final.weight'])
    print(parm['conv_final.bias'])
    '''

    test_dataset = dataset.DenoisingValDataset(opt)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = opt.test_batch_size, shuffle = False, num_workers = opt.num_workers, pin_memory = True)
    sample_folder = opt.save_name
    utils.check_path(sample_folder)

    psnr_sum, psnr_ave, ssim_sum, ssim_ave, time_sum, time_ave, eval_cnt = 0, 0, 0, 0, 0, 0, 0
    #np.set_printoptions(threshold=np.inf)
    
    # Count start time
    prev_time = time.time()
    
    # forward
    for i, (true_input, true_target, height_origin, width_origin) in enumerate(test_loader):
        prev_time = time.time()

        # To device
        if opt.no_gpu:
            true_input = true_input
            true_target = true_target
        else:
            true_input = true_input.cuda()
            true_target = true_target.cuda()            

        # Forward propagation
        with torch.no_grad():
            #print(true_input.size()) 
            fake_target, feature_map = generator(true_input, true_input)
        
        #print(fake_target.shape, true_input.shape)
        
        time_iter = time.time() - prev_time
        time_sum = time_sum + time_iter
        #prev_time = time.time()

        # Save
        img_list = [true_input, fake_target, true_target]
        name_list = ['in', 'pred', 'gt']
        sample_name = '%d' % (i+1)
        utils.save_sample_png(sample_folder = sample_folder, sample_name = '%d' % (i + 1), img_list = img_list, name_list = name_list, pixel_max_cnt = 255, height = height_origin, width = width_origin)
        
        # plot filter channel in grayscale
        
        #plt.imshow(feature_map[0][0].cpu().detach().numpy(), cmap='gray')
        #plt.savefig(sample_folder+'/'+ str(i+1) + '_featuremap.png')
        #fm_content=feature_map[0][0].cpu().detach().numpy()
        #print(fm_content)
        #utils.text_save(content = fm_content, filename = fm_file)
        
        # Evaluation
        #psnr_sum = psnr_sum + utils.psnr(cv2.imread(sample_folder + '/' + sample_name + '_' + name_list[1] + '.png').astype(np.float32), cv2.imread(sample_folder + '/' + sample_name + '_' + name_list[2] + '.png').astype(np.float32))
        img_pred_recover = utils.recover_process(fake_target, height = height_origin, width = width_origin)
        img_gt_recover = utils.recover_process(true_target, height = height_origin, width = width_origin)
        #psnr_sum = psnr_sum + utils.psnr(utils.recover_process(fake_target, height = height_origin, width = width_origin), utils.recover_process(true_target, height = height_origin, width = width_origin))
        psnr_sum = psnr_sum + utils.psnr(img_pred_recover, img_gt_recover)
        ssim_sum = ssim_sum + compare_ssim(img_gt_recover, img_pred_recover, multichannel = True, data_range = 255)
        
        eval_cnt = eval_cnt + 1
        print('The '+ str(i) +'-th iteration: sec: ' + str(time_iter))
        
    psnr_ave = psnr_sum / eval_cnt
    ssim_ave = ssim_sum / eval_cnt
    time_ave = time_sum / eval_cnt
    fps = 1/time_ave
    eval_file = "../data/evaluation_data.txt"
    
    psnr_content = opt.load_gname + ": psnr:" + str(psnr_ave) + ", ssim:" + str(ssim_ave) + ", fps: " + str(fps) + "\n"
    utils.text_save(content = psnr_content, filename = eval_file)
    
    
    


Namespace(add_noise=False, angle_aug=False, baseroot='../rainy_image_dataset/raindrop/test_b', crop=False, crop_size=512, geometry_aug=False, init_gain=0.02, init_type='xavier', load_gname='../models/models_SAUnet_SASkipnet_SAPerceptualVGG16SArelu53_raindrop/KPN_rainy_image_epoch200_bs8_generator.pth', mu=0, no_gpu=False, num_workers=1, resize=True, save_name='../results/models_SAUnet_SASkipnet_SAPerceptualVGG16SArelu53_raindrop_b', scale_max=1, scale_min=1, scale_size=1024, sigma=30, test_batch_size=1)
Generator are loaded!
The 0-th iteration: sec: 0.06690597534179688
The 1-th iteration: sec: 0.010944128036499023
The 2-th iteration: sec: 0.02066636085510254
The 3-th iteration: sec: 0.010607719421386719
The 4-th iteration: sec: 0.013096809387207031
The 5-th iteration: sec: 0.011222124099731445
The 6-th iteration: sec: 0.009732961654663086
The 7-th iteration: sec: 0.01047825813293457
The 8-th iteration: sec: 0.013664722442626953
The 9-th iteration: sec: 0.01100921630859375
The 10-th ite

The 163-th iteration: sec: 0.010576486587524414
The 164-th iteration: sec: 0.010434389114379883
The 165-th iteration: sec: 0.011331558227539062
The 166-th iteration: sec: 0.009940385818481445
The 167-th iteration: sec: 0.015018701553344727
The 168-th iteration: sec: 0.010915279388427734
The 169-th iteration: sec: 0.010466814041137695
The 170-th iteration: sec: 0.010612726211547852
The 171-th iteration: sec: 0.010456085205078125
The 172-th iteration: sec: 0.01073145866394043
The 173-th iteration: sec: 0.01066899299621582
The 174-th iteration: sec: 0.010050535202026367
The 175-th iteration: sec: 0.01039433479309082
The 176-th iteration: sec: 0.010940074920654297
The 177-th iteration: sec: 0.010414600372314453
The 178-th iteration: sec: 0.009602785110473633
The 179-th iteration: sec: 0.010508537292480469
The 180-th iteration: sec: 0.010320901870727539
The 181-th iteration: sec: 0.010408878326416016
The 182-th iteration: sec: 0.010253667831420898
The 183-th iteration: sec: 0.01065230369567